In [32]:
#pip install nltk

  Using cached nltk-3.6.5-py3-none-any.whl (1.5 MB)
  Using cached click-8.0.3-py3-none-any.whl (97 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
     |████████████████████████████████| 288 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [49]:
import pandas as pd
import re, string
import time
import json
from nltk import tokenize
import math

Setting up wiki

In [19]:
with open('Computed/wiki.json') as json_file:
    wiki_dict = json.load(json_file)
wiki = pd.read_csv('Computed/wiki.csv')[['content','title','id']]
print(wiki)

                                                   content  \
0        morocco saudi arabia relations moroccan saudi ...   
1        anthoni unit post office th anthoni unit post ...   
2        dumraon vidhan sabha constitu dumraon vidhan s...   
3        chaker khazaal chak khazaal born 28 septemb 19...   
4        vicent pascual pastor vicent pascual pastor al...   
...                                                    ...   
1286259  toppr toppr product haygot educ privat limit f...   
1286260  outlin education th outlin overview topic guid...   
1286261  raell padamsee rael padamse indian theatr pers...   
1286262  megan callaway megan callaway founder archer s...   
1286263  dian reay dian reay sociologist professor educ...   

                                     title       id  
0           Morocco–Saudi Arabia relations        1  
1        Anthony United States Post Office        2  
2        Dumraon Vidhan Sabha constituency        3  
3                           Chaker Khaz

Splitting Documents into sentences

In [22]:
def splitDoc(s):
    sentence_spliter = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', str (s))
    return sentence_spliter

df1 = wiki
df1['content'] = wiki['content'].apply(splitDoc)

In [25]:
df1['content'].iloc[0]

['morocco saudi arabia relations moroccan saudi arabian relat refer current histor relat morocco saudi arabia morocco embassi riyadh saudi arabia embassi rabat both nation long tradit friendli relat base histor common morocco saudi arabia monarch rule respect famili alaouit saud adher sunni islam fear grow iranian shiit influenc relationship excel morocco saudi arabia step curb iranian influenc arab morocco moder approach iran saudi arabia cautiou hostil iran saudi arabia support morocco disput western sahara recogn legitimaci sahrawi arab democrat republ arab leagu saudi arabia help fund project region support moroccan author dure current yemeni civil war morocco particip anti houthi coalit led saudi arabia 6 plane 1 500 troop morocco neutral stanc conflict hope resolut benefit qatar saudi arabia particip morocco anti houthi campaign morocco offer mediat slow tension gulf arab']

In [36]:
sample = 'morocco saudi arabia relations moroccan. saudi arabian relat S. refer current histor. relat morocco saudi arabia morocco embassi riyadh saudi arabia embassi rabat both nation long tradit friendli relat base histor common morocco saudi arabia monarch rule respect famili alaouit saud adher sunni islam fear grow iranian shiit influenc relationship excel morocco saudi arabia. step curb 3.14 iranian influenc arab morocco moder approach iran saudi arabia cautiou hostil W. iran saudi arabia support morocco disput western sahara recogn legitimaci sahrawi arab democrat republ arab leagu saudi arabia help fund project region support moroccan author dure current yemeni civil war morocco particip anti houthi coalit led saudi arabia 6 plane 1 500 troop morocco neutral stanc conflict hope resolut benefit qatar saudi. arabia particip morocco anti houthi campaign morocco offer mediat slow tension gulf arab.'
tokenize.sent_tokenize(sample)

['morocco saudi arabia relations moroccan.',
 'saudi arabian relat S. refer current histor.',
 'relat morocco saudi arabia morocco embassi riyadh saudi arabia embassi rabat both nation long tradit friendli relat base histor common morocco saudi arabia monarch rule respect famili alaouit saud adher sunni islam fear grow iranian shiit influenc relationship excel morocco saudi arabia.',
 'step curb 3.14 iranian influenc arab morocco moder approach iran saudi arabia cautiou hostil W. iran saudi arabia support morocco disput western sahara recogn legitimaci sahrawi arab democrat republ arab leagu saudi arabia help fund project region support moroccan author dure current yemeni civil war morocco particip anti houthi coalit led saudi arabia 6 plane 1 500 troop morocco neutral stanc conflict hope resolut benefit qatar saudi.',
 'arabia particip morocco anti houthi campaign morocco offer mediat slow tension gulf arab.']

NEW TFIDF FOR SENTENCES

TF

In [60]:
def sentenceTF(word, sentences):
    content = []
    wordCounts = []
    sentenceCount = []
    wordCount = 0
    sentence = tokenize.sent_tokenize(sentences)
    for s in sentence:
        wordCount = 0
        if word in s.split(" "):
            wordCount += 1
        content.append(s)
        wordCounts.append(wordCount)
        sentenceCount.append(len(s.split(" ")))
    df2 = pd.DataFrame(content)
    df2['WordCounts'] = wordCounts
    df2['NumWords'] = sentenceCount
    df2['TF'] = (df2['WordCounts']/df2['NumWords'])
    return df2
word = 'morocco'
sample = 'morocco saudi arabia relations moroccan. saudi arabian relat S. refer current histor. relat morocco saudi arabia morocco embassi riyadh saudi arabia embassi rabat both nation long tradit friendli relat base histor common morocco saudi arabia monarch rule respect famili alaouit saud adher sunni islam fear grow iranian shiit influenc relationship excel morocco saudi arabia. step curb 3.14 iranian influenc arab morocco moder approach iran saudi arabia cautiou hostil W. iran saudi arabia support morocco disput western sahara recogn legitimaci sahrawi arab democrat republ arab leagu saudi arabia help fund project region support moroccan author dure current yemeni civil war morocco particip anti houthi coalit led saudi arabia 6 plane 1 500 troop morocco neutral stanc conflict hope resolut benefit qatar saudi. arabia particip morocco anti houthi campaign morocco offer mediat slow tension gulf arab.'

df2 = sentenceTF(word, sample)
print(df2)

                                                   0  WordCounts  NumWords  \
0           morocco saudi arabia relations moroccan.           1         5   
1       saudi arabian relat S. refer current histor.           0         7   
2  relat morocco saudi arabia morocco embassi riy...           1        42   
3  step curb 3.14 iranian influenc arab morocco m...           1        67   
4  arabia particip morocco anti houthi campaign m...           1        13   

         TF  
0  0.200000  
1  0.000000  
2  0.023810  
3  0.014925  
4  0.076923  


sentence IDF

In [61]:
nw = 0
idf = 0
def sentenceIDF(WordCount):
    numSentences = len(df2)
    global nw
    global idf
    if int(WordCount) > 0:
        nw += 1
    
    idf = math.log2(numSentences/nw)
    
df2['WordCounts'].apply(sentenceIDF)
print(idf)

0.32192809488736235


In [62]:
def getSentenceTFIDF(word):
    return sentenceTF(word, sample)['TF'] * (idf)

print (getSentenceTFIDF('morocco'))

0    0.064386
1    0.000000
2    0.007665
3    0.004805
4    0.024764
Name: TF, dtype: float64


DENOMINATOR

In [66]:
def firstHalf(sentence):
    firsthalf = math.sqrt(len(sentence)*1)
    return (firsthalf)

totalTFIDF = 0
def secondHalf(query):
    global totalTFIDF
    for word in query.split(" "):
        totalTFIDF += getSentenceTFIDF(word)**2
    return (math.sqrt(totalTFIDF))

query = "morroco saudi"
secondHalf(query)

TypeError: cannot convert the series to <class 'float'>

COSINE SIMULARITY

In [ ]:
def cosineSimularity(word, docID):
    for doc in df:
        splitDoc();
        for sectence in doc:
            sentence 